### Zaawansowane Metody Uczenia Maszynowego

*Środowisko z poprzednich zajęć*

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
diamonds = sns.load_dataset("diamonds")
diamonds = pd.DataFrame(diamonds)

diamonds = diamonds[diamonds["cut"] != "Very Good"]

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.linear_model import LogisticRegression

preprocesor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore"), ["color", "clarity"]),
        ("power_transform", PowerTransformer(), ["price", "carat"]),
        ("scale", StandardScaler(), ["x", "y", "z", "depth", "table", "price", "carat"])
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocess", preprocesor),
        ("model", LogisticRegression(max_iter=2000))
    ]
)


y = diamonds.cut
X = diamonds.drop(["cut"], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=pipeline.classes_)
disp.plot()

In [ ]:
y_score = pipeline.predict_proba(X_test)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

In [ ]:
class_of_interest = "Good"
class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
class_id

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

display = RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    color="darkorange",
    plot_chance_level=True,
)
_ = display.ax_.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="One-vs-Rest ROC curves:\nGood vs Other",
)

https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

#### Laboratorium 3
*Inspirowane materiałami Kamila Kmity.*

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
rng = np.random.default_rng(2024)

### Zadanie 1
----
Wygeneruj $200$ obserwacji pseudolosowych takich, że

\begin{equation}
Y = \beta_0 + \beta_1 \cdot X_1 + \beta_2 \cdot X_2 + \beta_3 \cdot X_3 + \epsilon,
\end{equation}

gdzie

* $\beta = (50, -2, 2.25, 3)$,
* $X_1 \sim \mathcal{N}(98, 10)$,
* $X_2 \sim \mathcal{N}(50, 5)$,
* $X_3 \sim \mathcal{N}(231, 12)$,
* $\epsilon \sim \mathcal{N}(0, 4.75)$.

[Jak losować liczby w `numpy`.](https://numpy.org/doc/stable/reference/random/generator.html)


### Regresja liniowa

Cel: znalezienie optymalnego wektora parametrów
 $\beta = (\beta_0, \beta_1, \ldots, \beta_p)^T$.

W tym celu definiujemy funkcję ryzyka $L(\beta)$ i szukamy

$$
\begin{equation}
\hat{\beta} = \underset{\beta}{\text{arg min}} \quad
L(\beta)
\end{equation}
$$

Powszechnie stosowana funkcja ryzyka
([o przydatnych statystycznie właściwościach](https://en.wikipedia.org/wiki/Ordinary_least_squares#Properties)) to *Mean Squared Error* MSE

$$
\begin{align}
L(\beta) &= \text{MSE}(\beta; X, y)\\
&= \sum\limits_{i=1}^n (\hat{y}_i - y_i)^2 \\
&= \sum\limits_{i=1}^n ({\beta} \mathbf{x}_i - y_i)^2
\qquad \text{where }\mathbf{x}_i \in R^p
\\
&= {\lVert X {\beta} - Y \rVert}^2
= (X {\beta} - Y)^T (X {\beta} - Y)
\end{align}
$$

Jak wyznaczyć $\hat{\beta}$?

### Zadanie 2
----
Zaimplementuj klasę `Regression` z metodą `fit` i dopasuj model regresji liniowej do wysymulowanych danych `data`.

Szkielet rozwiązania

```python
class Regression:
    def __init__(self, ...)
        self.weights = None
        ...

    def fit(self, X, y):
        ...
```

### Zadanie 3
---
W powyższym przykładzie nie zawarliśmy wyrazu wolnego *intercept* . Możemy dołączyć wyraz wolny poprzez rozszerzenie macierzy:

$X := [\mathbf{1} | X]$,

Zmodyfikuj metody `__init__` oraz `fit` klasy `Regression` tak, żeby użytkownik miał wybór: użyć intercept lub nie.

Szkielet rozwiązania

```python
class Regression:
    def __init__(self, intercept=False)
        self.weights = None
        self.intercept = intercept

    def fit(self, X, y):
        if self.intercept:
            pass
        pass
```

## Reprezentacja regresji liniowej jako prostej sieci neuronowej

*Note*
[Zapoznaj się z krótkim wprowadzeniem do `Keras`.](https://keras.io/about/)

### Zadanie 4
Zaimplementuj model regresji używając sieci neuronowej.

W modelu:
* $x$ to wektor kolumnowy `[[X1], [X2], [X3]]` (czyli macierz `np.array` o wymiarze 3x1),
* $W$ = `[[w1, w2, w3]]` to macierz o wymiarze 1x3,
* $b \in R$,
* $z=Wx + b$ to warstwa typu `tf.keras.layers.Dense`,
* funkcja aktywacji $\sigma = Id$, co zapewnia nam model liniowy.

Utwórz model `model1` korzystając z biblioteki `Keras`, korzystając ze szkieletu poniżej. Korzystaj z *Sequential API*.

```python
model1 = ...

model1.add(tf.keras.Input(shape=...))

model1.add(
    tf.keras.layers.Dense(
        ..., 
        activation=...
    )
)
```